In [1]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import numpy as np
import pandas as pd
import my_globals
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from utils import get_sub_featured_datasets


In [3]:
DATA_PATH = "/".join([my_globals.DATA_DIR, my_globals.MAIN_DATA_NAME])
data = get_sub_featured_datasets(size = 30000, random_seed=4)
data.head(5)

C:\Users\linru\AppData\Roaming\Python\Python39\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,index,target,ids,date,user,text,weekday_Mon,weekday_Tue,weekday_Wed,weekday_Thu,...,tfidf_ð¾ð²ðµñ,tfidf_ð¾ñ,tfidf_øª,tfidf_øªø,tfidf_øªø³ø,tfidf_øªøµø,tfidf_ø³ø¹ù,tfidf_øµù,tfidf_ø¹,tfidf_ø¹ù
0,691356,0,2252217215,Sat Jun 20 05:45:17 PDT 2009,graciekate,@mewize I wish I quit signing up for weekends,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,572155,0,2209169022,Wed Jun 17 10:05:19 PDT 2009,Maymay10293,missing my phone,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,331113,0,2012552114,Tue Jun 02 20:51:28 PDT 2009,marygowens,@DonnieWahlberg my sister @jennwest630 is your...,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,389597,0,2054486341,Sat Jun 06 07:27:32 PDT 2009,ccolwin,@RicoDanielle It's so hard to call your hotlin...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,385300,0,2053577112,Sat Jun 06 05:02:06 PDT 2009,WuppieDesu,"Tweet no. 100! - Bad news though, one of my fr...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
features_col = [
    col for col in data.columns 
    if (
        (col.startswith("weekday") or 
        #  col.startswith("count") or 
         col.startswith("tfidf")
         ) and col != "target")
]

other_features = ["exclaim_freq", "mention_count", "cap_freq"]
features_col += other_features
print("Non-token features:")
print([col for col in features_col if not (col.startswith("count") or col.startswith("tfidf"))])

XX = data[features_col]
yy = data[["target"]]
X_train, X_test, y_train, y_test = train_test_split(XX, yy, test_size=0.2)

Non-token features:
['weekday_Mon', 'weekday_Tue', 'weekday_Wed', 'weekday_Thu', 'weekday_Fri', 'weekday_Sat', 'weekday_Sun', 'exclaim_freq', 'mention_count', 'cap_freq']


## Benchmarking with BernoulliNB

In [5]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

bnb = BernoulliNB()
X_input = X_train
bnb.fit(X_input, y_train)
y_pred = bnb.predict(
    X_test
)

def assess(y_true, y_pred):
    print("confusion matrix:")
    print(confusion_matrix(y_true, y_pred))
    print()
    print("accuracy_score:")
    print(accuracy_score(y_true, y_pred))
    print()
    print("classification report:")
    print(classification_report(y_true,y_pred))
    print()

# print("confusion matrix:")
# print(confusion_matrix(y_test, y_pred))
# print()
# print("accuracy_score:")
# print(accuracy_score(y_test, y_pred))
# print()
# print("classification report:")
# print(classification_report(y_test,y_pred))
# print()
assess(y_test, y_pred)

c:\Users\linru\Anaconda3\envs\DL\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


confusion matrix:
[[2239  691]
 [ 871 2199]]

accuracy_score:
0.7396666666666667

classification report:
              precision    recall  f1-score   support

           0       0.72      0.76      0.74      2930
           4       0.76      0.72      0.74      3070

    accuracy                           0.74      6000
   macro avg       0.74      0.74      0.74      6000
weighted avg       0.74      0.74      0.74      6000




In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

X_input = X_train

param_grid = {
    'n_estimators': [100, 200, 400],
    'max_features': [
        'sqrt', 
        'log2', 
        # None
    ],
    'max_depth': [10, 50, 100],
    # 'max_leaf_nodes': [3, 6, 9],
}


grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid, verbose=4)
grid_search.fit(X_input, y_train)

rfc = grid_search.best_estimator_
print(rfc)
rfc.fit(X_input, y_train)
y_pred = rfc.predict(X_test)
print(accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END max_depth=10, max_features=sqrt, n_estimators=100;, score=0.691 total time=  12.3s
[CV 2/5] END max_depth=10, max_features=sqrt, n_estimators=100;, score=0.683 total time=  12.4s
[CV 3/5] END max_depth=10, max_features=sqrt, n_estimators=100;, score=0.697 total time=  12.8s
[CV 4/5] END max_depth=10, max_features=sqrt, n_estimators=100;, score=0.684 total time=  12.7s
[CV 5/5] END max_depth=10, max_features=sqrt, n_estimators=100;, score=0.691 total time=  12.0s
[CV 1/5] END max_depth=10, max_features=sqrt, n_estimators=200;, score=0.695 total time=  22.4s
[CV 2/5] END max_depth=10, max_features=sqrt, n_estimators=200;, score=0.702 total time=  20.7s
[CV 3/5] END max_depth=10, max_features=sqrt, n_estimators=200;, score=0.699 total time=  20.4s
[CV 4/5] END max_depth=10, max_features=sqrt, n_estimators=200;, score=0.697 total time=  19.7s
[CV 5/5] END max_depth=10, max_features=sqrt, n_estimators=200;, score=0.69

In [9]:
rfc.fit(X_input, y_train)
y_pred = rfc.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.742


In [10]:
def assess(y_true, y_pred):
    print("confusion matrix:")
    print(confusion_matrix(y_true, y_pred))
    print()
    print("accuracy_score:")
    print(accuracy_score(y_true, y_pred))
    print()
    print("classification report:")
    print(classification_report(y_true,y_pred))
    print()

assess(y_test, y_pred)

confusion matrix:
[[2190  740]
 [ 808 2262]]

accuracy_score:
0.742

classification report:
              precision    recall  f1-score   support

           0       0.73      0.75      0.74      2930
           4       0.75      0.74      0.75      3070

    accuracy                           0.74      6000
   macro avg       0.74      0.74      0.74      6000
weighted avg       0.74      0.74      0.74      6000




In [8]:
import matplotlib.pyplot as plt
import pandas as pd

feature_names = rfc.feature_names_in_
importances = rfc.feature_importances_
idx = np.argsort(importances)
feature_names[idx][-100:]

array(['tfidf_bored', 'tfidf_hi', 'tfidf_phone', 'tfidf_would',
       'tfidf_wait', 'tfidf_school', 'tfidf_excited', 'tfidf_gone',
       'tfidf_hope', 'tfidf_amazing', 'tfidf_never', 'tfidf_feeling',
       'tfidf_best', 'tfidf_tomorrow', 'tfidf_it', 'tfidf_much',
       'tfidf_cry', 'tfidf_missed', 'tfidf_glad', 'tfidf_week',
       'tfidf_think', 'tfidf_but', 'tfidf_doe', 'tfidf_quot', 'tfidf_see',
       'tfidf_welcome', 'tfidf_lost', 'tfidf_oh', 'tfidf_home',
       'tfidf_working', 'tfidf_yay', 'tfidf_cool', 'tfidf_fun',
       'tfidf_time', 'tfidf_though', 'tfidf_rain', 'tfidf_one',
       'tfidf_well', 'tfidf_tired', 'tfidf_morning', 'tfidf_back',
       'tfidf_yes', 'tfidf_watching', 'tfidf_night', 'tfidf_last',
       'tfidf_could', 'tfidf_today', 'tfidf_know', 'tfidf_like',
       'tfidf_missing', 'tfidf_poor', 'tfidf_sleep', 'tfidf_got',
       'tfidf_no', 'tfidf_not', 'weekday_Fri', 'tfidf_my', 'weekday_Tue',
       'tfidf_need', 'tfidf_hey', 'tfidf_haha', 'tfidf_going',
